In [423]:
import pandas as pd
from web3 import Web3
from hexbytes import HexBytes
import ast
import rlp
from rlp.sedes import Binary, big_endian_int, binary, List
from eth_utils import to_bytes, to_hex, int_to_big_endian, decode_hex
import sys
import os
import dotenv

dotenv.load_dotenv()
sys.path.append("../../helper_functions")
import clickhouse_utils as ch
import duneapi_utils as du
sys.path.pop()

client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

In [424]:
# Test transaction receipt
from web3 import Web3
op_rpc = os.getenv("OP_PUBLIC_RPC")
w3 = Web3(Web3.HTTPProvider(op_rpc))

tx_test = '0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b89538aa232013d91edcb926'
tx = w3.eth.get_transaction(tx_test)
txr = w3.eth.get_transaction_receipt(tx_test)
# # txraw = w3.eth.get_raw_transaction(tx_test)
print(tx)
# print(txr)
# # print(txraw)

AttributeDict({'blockHash': HexBytes('0x694fb0156a5719a8ee75ebf2568e8a8d55899db9312127f0d4fe5001c06ac397'), 'blockNumber': 120731426, 'chainId': 10, 'from': '0x899e837095a0F3CC62FB05998559Df90F26A1F46', 'gas': 69534, 'gasPrice': 4307539, 'hash': HexBytes('0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b89538aa232013d91edcb926'), 'input': HexBytes('0xa9059cbb0000000000000000000000007a6e883eec3dd33528115637ea01b3b64e2f58490000000000000000000000000000000000000000000000009e34ef99a7740000'), 'nonce': 206, 'r': HexBytes('0x6727a53c0972c55923242cea052dc4e1105d7b65c91c442e2741440965eac357'), 's': HexBytes('0x0a8e71aea623adb7b5562fb9a779634f3b84dad7be1e1f22caaa640db352a6ff'), 'to': '0xdC6fF44d5d932Cbd77B52E5612Ba0529DC6226F1', 'transactionIndex': 4, 'type': 0, 'v': 55, 'value': 0})


In [425]:
# may not sufficent due to missing transaction signature fields

# Get L2 Txs from Clickhouse / Goldsky
query_by_day = '''
        SELECT 10 as chain_id, nonce, gas, max_fee_per_gas, max_priority_fee_per_gas, gas_price,
                to_address as to, value, input, block_timestamp, block_number, hash, receipt_gas_used
        FROM op_transactions
        WHERE gas_price > 0
        AND hash = '0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b89538aa232013d91edcb926'
        AND block_number = 120731426

        limit 1

        SETTINGS max_execution_time = 7000
'''
result_df = client.query_df(query_by_day)
                            
result_df['access_list'] = '[]'
result_df['access_list'] = result_df['access_list'].apply(ast.literal_eval)
result_df['r'] = '0x6727a53c0972c55923242cea052dc4e1105d7b65c91c442e2741440965eac357'
result_df['s'] = '0x0a8e71aea623adb7b5562fb9a779634f3b84dad7be1e1f22caaa640db352a6ff'
result_df['v'] = '55'


In [426]:
result_df

,chain_id,nonce,gas,max_fee_per_gas,max_priority_fee_per_gas,gas_price,to,value,input,block_timestamp,block_number,hash,receipt_gas_used,access_list,r,s,v
0,10,206,69534,0,0,4307539,b'0xdc6ff44d5d932cbd77b52e5612ba0529dc6226f1',0,0xa9059cbb0000000000000000000000007a6e883eec3d...,2024-05-30 09:33:49,120731426,b'0xcea81f2e836a37b38ba82afd37e6f66c02e348e7b8...,34408,[],0x6727a53c0972c55923242cea052dc4e1105d7b65c91c...,0x0a8e71aea623adb7b5562fb9a779634f3b84dad7be1e...,55


In [427]:
# Process "to" field

to_field = result_df['to'][0].decode('utf-8')
v_field = result_df['v'][0]

# print(f"to_field (original): {to_field}")

# to_bytes = bytes.fromhex(to_field[2:])

# print(f"to_bytes (result): {to_bytes}")

print( int_to_big_endian(int(v_field)) )  # Convert v to a bytes object
print(int(v_field).to_bytes(1, byteorder='big'))

b'7'
b'7'


In [441]:
def process_and_encode_transaction(row):
    try:
        # Process "to" field
        to_field = row['to'].decode('utf-8')
        to_field = result_df['to'][0].decode('utf-8')
        to_bytes = bytes.fromhex(to_field[2:])

        print(f"input (before): {row['input']}")
        print(f"input (after): {bytes.fromhex(row['input'][2:])}")
        print(f"input length: {len(bytes.fromhex(row['input'][2:]))}")
        # Prepare transaction parameters
        tx_params = {
            'nonce': int_to_big_endian(row['nonce']),
            'gasPrice': int_to_big_endian(row['gas_price']),
            'gas': int_to_big_endian(row['gas']),
            'to': to_bytes,
            'value': int_to_big_endian(row['value']) if row['value'] != 0 else b'',  # Encode value as byte array if 0
            'input': bytes.fromhex(row['input'][2:]),
            'v': int_to_big_endian(int(row['v'])),  # Convert v to a bytes object
            'r': bytes.fromhex(row['r'][2:]),
            's': bytes.fromhex(row['s'][2:])
        }

        # Print transaction parameters for debugging
        for key, value in tx_params.items():
            print(f"{key}: {value}, {type(value)}")

        # Prepare the transaction fields for RLP encoding
        transaction = [
            tx_params['nonce'],
            tx_params['gasPrice'],
            tx_params['gas'],
            tx_params['to'],
            tx_params['value'],
            tx_params['input'],
            tx_params['v'],
            tx_params['r'],
            tx_params['s']
        ]
        print(transaction)

        # Encode the entire transaction
        encoded_tx = rlp.encode(transaction)
        encoded_tx_hex = "0x" + encoded_tx.hex()
        return encoded_tx_hex, len(encoded_tx)

    except (ValueError, TypeError, UnicodeDecodeError) as e:
        print("Error:", e)
        print("Failed Transaction Info:")
        print(row)
        return None, None

In [442]:
correct_raw_tx_hash = '0xf8a981ce8341ba5383010f9e94dc6ff44d5d932cbd77b52e5612ba0529dc6226f180b844a9059cbb0000000000000000000000007a6e883eec3dd33528115637ea01b3b64e2f58490000000000000000000000000000000000000000000000009e34ef99a774000037a06727a53c0972c55923242cea052dc4e1105d7b65c91c442e2741440965eac357a00a8e71aea623adb7b5562fb9a779634f3b84dad7be1e1f22caaa640db352a6ff'

In [443]:
# Assuming result_df is your DataFrame with the transaction data
result_df[['encoded_transaction', 'len_encoded_transaction']] = result_df.apply(
    lambda row: process_and_encode_transaction(row),
    axis=1,
    result_type='expand'
)
rawtx = result_df['encoded_transaction'][0]
print(f"Encoded transaction: 0x{rawtx}")
print(len(rawtx))


input (before): 0xa9059cbb0000000000000000000000007a6e883eec3dd33528115637ea01b3b64e2f58490000000000000000000000000000000000000000000000009e34ef99a7740000
input (after): b'\xa9\x05\x9c\xbb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00zn\x88>\xec=\xd35(\x11V7\xea\x01\xb3\xb6N/XI\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x9e4\xef\x99\xa7t\x00\x00'
input length: 68
nonce: b'\xce', <class 'bytes'>
gasPrice: b'A\xbaS', <class 'bytes'>
gas: b'\x01\x0f\x9e', <class 'bytes'>
to: b'\xdco\xf4M]\x93,\xbdw\xb5.V\x12\xba\x05)\xdcb&\xf1', <class 'bytes'>
value: b'', <class 'bytes'>
input: b'\xa9\x05\x9c\xbb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00zn\x88>\xec=\xd35(\x11V7\xea\x01\xb3\xb6N/XI\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x9e4\xef\x99\xa7t\x00\x00', <class 'bytes'>
v: b'7', <class 'bytes'>
r: b"g'\xa5<\tr\xc5Y#$,\xea\x05-\xc4\xe1\x10]{e\xc9\x1cD.'AD\te\xea\xc3W", <class 'byt

In [444]:
rawtx = result_df['encoded_transaction'][0]
# print(rawtx)
print(len(rawtx))
print(len(correct_raw_tx_hash))


342
344
